In [ ]:
import gust
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

from scipy.spatial.distance import squareform

In [ ]:
torch.set_default_tensor_type('torch.cuda.FloatTensor')

A, X, _, z = gust.load_dataset('cora_ml').standardize().unpack()
adj = torch.FloatTensor(A.toarray()).cuda()

In [ ]:
N = A.shape[0]
D = 32

Z = nn.Parameter(torch.empty(N, D).normal_(std=0.1))
x = nn.Parameter(torch.empty(N, D).normal_(std=0.1))

opt = torch.optim.Adam([Z], lr=1e-2)
e1, e2 = A.nonzero()

In [ ]:
def L0(Z, A):    
    return torch.FloatTensor(A.toarray()).cuda()

In [41]:
def L1(Z, A):
    #L=D^-1A
    degreenp=A.sum(axis=1)
    degree = torch.from_numpy(degreenp).cuda().view(-1)
    degreematrix = torch.zeros(N,N)
    degreematrix[np.diag_indices(N)]=degree
    invdegree = torch.inverse(degreematrix)
    adj = torch.FloatTensor(A.toarray()).cuda()
    return torch.matmul(invdegree, adj)

In [50]:
def L2(Z, A):
    degreenp=A.sum(axis=1)
    degree = torch.from_numpy(degreenp).cuda().view(-1)
    degreematrix = torch.zeros(N,N)
    degreematrix[np.diag_indices(N)]=degree
    sqrtdegree = np.linalg.sqrtm(degreematrix)
    invdegree = torch.inverse(sqrtdegree)
    adj = torch.FloatTensor(A.toarray()).cuda() 
    return torch.matmul(invdegree, adj, invdegree)

In [51]:
L2(Z, A)

AttributeError: module 'numpy.linalg' has no attribute 'sqrtm'

In [4]:
def sig(Z, b=0.1, eps=1e-8): 
    dist = torch.matmul(Z,Z.T) +b
    sigdist = 1/(1+torch.exp(dist+eps)+eps)
    logsigdist = torch.log(sigdist+eps)
    pos_term = logsigdist[e1,e2]
    neg_term = torch.log(1-sigdist)
    neg_term[np.diag_indices(N)] = 0.0
    
    return -(pos_term.sum() + neg_term.sum()) / Z.shape[0]**2

In [5]:
def dist(Z, eps=1e-5):
    dist = ((Z[:, None] - Z[None, :]).pow(2.0).sum(-1) + eps).sqrt()
    neg_term = torch.log(-torch.expm1(-dist) + eps)
    neg_term[np.diag_indices(N)] = 0.0
    pos_term = -dist[e1, e2]   
    neg_term[e1, e2] = 0.0
    
    return -(pos_term.sum() + neg_term.sum()) / Z.shape[0]**2

In [6]:
def exp(Z, eps=1e-8):
    Z_abs = torch.FloatTensor.abs(Z)
    dist = -torch.matmul(Z_abs,Z_abs.T)
    neg_term=dist
    neg_term[np.diag_indices(N)]=0.0
    expdist=torch.exp(dist)
    logdist=torch.log(1-expdist+eps)
    pos_term = logdist[e1,e2]
    
    return -(pos_term.sum() + neg_term.sum()) / Z.shape[0]**2

In [7]:
def kl(L, Z, eps=1e-8):
    #P=softmax(ZZ^T)
    dist=torch.matmul(Z,Z.T)
    sigdist = 1/(1+torch.exp(dist+eps)+eps)
    logsigdist = torch.log(sigdist+eps)
    losses = T*logsigdist
    return losses.sum()

In [8]:
for epoch in range(100):
    opt.zero_grad()
    loss = kl(Z)
    loss.backward()
    opt.step()
    print(loss.item())

TypeError: kl() missing 1 required positional argument: 'Z'